# Metoda končnih volumnov za konvekcijsko difuzijske probleme / <font color='gray'>The Finite Volume Method for Convection-Diffusion Problems</font>

Avtor: / <font color='gray'> Author: </font> Boštjan Mavrič, Matjaž Zadnik

Povzeto po primeru: / <font color='gray'> Adapted from the case: </font>
H.K. Versteeg, M.L. Malalasekera, An Introduction to Computational Fluid Dynamics, The Finite Volume Method, Second Edition, Prentice Hall, Harlow, 2007 (104 - 125).

## Definicija naloge / <font color='gray'>Problem definition</font>

Ohranjena količina $\phi$ se v enodimenzionalni domeni prenaša z robnimi pogoji  $\phi_0=0$ pri $x=1$ in $\phi_L= 0$ pri $x=L$. Reši stacionarno konvekcijsko-difuzijsko enačbo za različne vrednosti hitrosti $u$:
    
$$\frac{\partial }{\partial x}(\rho u \phi)=\frac{\partial }{\partial x}\big( \Gamma \frac{\partial }{\partial x} \phi \big)$$
    
Podatki:
    
- Dolžina domene: $L = 1 m$
- Gostota: $\rho = 1 \frac{kg}{m^3}$
- Prevodnost: $\Gamma = 0.1\frac{kg}{ms}$

Rešitev dobljeno z MKV primerjaj z analitično rešitvijo:

$$\frac{\phi - \phi_0}{\phi_L - \phi_0}=\frac{\exp(\rho u x/\gamma)-1}{\exp(\rho u L/\gamma)-1}$$


<p><font color='gray'>

A conserved property $\phi$ is transported through the one-dimensional domain with boundary conditions $\phi_0=0$ at $x=1$ and $\phi_L= 0$ at $x=L$. Solve the stationary convection-diffusion equation for different values of velocity $u$:
    
$$\frac{\partial }{\partial x}(\rho u \phi)=\frac{\partial }{\partial x}\big( \Gamma \frac{\partial }{\partial x} \phi \big)$$.
    
The data:
    
- Domain length: $L = 1 m$
- Density: $\rho = 1 \frac{kg}{m^3}$
- Conductivity: $\Gamma = 0.1\frac{kg}{ms}$
    
Compare the solution obtained by FVM with the analytical solution:

$$\frac{\phi - \phi_0}{\phi_L - \phi_0}=\frac{\exp(\rho u x/\Gamma)-1}{\exp(\rho u L/\Gamma)-1}$$.

</font>
</p>

In [ ]:
# -*- coding: utf-8 -*-

#advection_central_difference, a demonstration of FVM for transport equation in 1D
#Copyright (C) 2020  Boštjan Mavrič, Matjaž Zadnik
#
#This program is free software: you can redistribute it and/or modify
#it under the terms of the GNU General Public License as published by
#the Free Software Foundation, either version 3 of the License, or
#(at your option) any later version.
#
#This program is distributed in the hope that it will be useful,
#but WITHOUT ANY WARRANTY; without even the implied warranty of
#MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#GNU General Public License for more details.
#
#You should have received a copy of the GNU General Public License
#along with this program.  If not, see <http://www.gnu.org/licenses/>.

from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual



def TDMAsolver(a, b, c, d):
        '''
        TDMA solver, a b c d can be NumPy array type or Python list type.
        refer to http://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm
        and to http://www.cfd-online.com/Wiki/Tridiagonal_matrix_algorithm_-_TDMA_(Thomas_algorithm)
        '''
        nf = len(d) # number of equations
        ac, bc, cc, dc = map(np.array, (a, b, c, d)) # copy arrays
        for it in range(1, nf):
            mc = ac[it-1]/bc[it-1]
            bc[it] = bc[it] - mc*cc[it-1] 
            dc[it] = dc[it] - mc*dc[it-1]

        xc = bc
        xc[-1] = dc[-1]/bc[-1]

        for il in range(nf-2, -1, -1):
            xc[il] = (dc[il]-cc[il]*xc[il+1])/bc[il]

        return xc

def CD_advection(N, u):
    # Transport v 1D z advekcijo in difuzijo / 1D advection-diffusion transport

    # Robni pogoji / boundary conditions
    fiA=1 # pri / at x=0
    fiB=0 # pri / at x=L

    # Snovni parametri / material parameters
    L=1 # [m] dolžina odseka / domain length
    rho=1 # [kg/m^3] gostota transportirane količine / density 
    Gama=0.1 # [kg/ms] posplošen koeficient prevodnosti / conductivity

    # Diskretizacija / discretization details
    dx=L/N # [m] krajevni korak / spatial step size


    # mrežne točke od 2 do N-1 / mesh points from 2 to N-1
    F=rho*u
    D=Gama/dx 
    Pe=F/D
    
    aE=D-F/2
    aW=D+F/2
    aP=aE+aW
    Su=0

    # Matrika sistema brez upoštevanja robnih pogojev / System matrix without the boundary conditions
    d=aP*np.ones(N) # diagonala / diagonal entries
    s=-aW*np.ones(N-1) # spodnja obdiagonala / lower diagonal
    z=-aE*np.ones(N-1) # zgornja obdiagonala / upper diagonal
    b=Su*np.ones(N) # vektor vrednosti izvorov / vector of source terms


    # levi rob (točka 1) / left boundary (point 1)
    aE=D-F/2
    aW=0
    Sp=-(2*D+F)
    Su=(2*D+F)*fiA
    aP=aE+aW-Sp

    d[0]=aP
    b[0]=Su

    # Desni rob (točka N) / right boundary (point N)
    aE=0
    aW=D+F/2
    Sp=-(2*D-F)
    Su=(2*D-F)*fiB
    aP=aE+aW-Sp

    d[N-1]=aP
    b[N-1]=Su

    # Rešimo sistem enačb / Solve the system of equations
    fi=TDMAsolver(s, d, z, b)

    # krajevna mreža / Spatial mesh
    x_tab=np.array([dx*i for i in range(N)])
    x_tab=x_tab+dx/2
    
    def analiticFi(x):
        'Točna rešitev'
        return fiA+(fiB-fiA)*(np.exp(rho*u*x/Gama)-1)/(np.exp(rho*u*L/Gama)-1)

    # Graf phi profila / Plot the profile of phi
    y_values=analiticFi(x_tab)
    eps_rel=np.linalg.norm(y_values-fi)/np.linalg.norm(y_values)
    
    title_string="Central difference scheme \n Pe=%.2f    Relative $L_2$ error= %.2e" % ( Pe, eps_rel)
    plt.plot(x_tab,fi,'bo',label='FVM')
    x_tab1=np.linspace(0,N*dx,1000)
    plt.plot(x_tab1,analiticFi(x_tab1),'b-',label='analytical')
    plt.xlim(0,N*dx)
    plt.xlabel('position $x$ (m)')
    plt.ylabel('$\Phi$ ')   
    plt.legend(loc='best')
    plt.title(title_string) 
    plt.show()
    

In [ ]:
#Pripravi UI / generate UI
N_slider = widgets.IntSlider(min=5, max=50, step=5, value=5)
u_slider = widgets.FloatSlider(min=-10, max=10, step=0.5, value=2.5)

w=widgets.interactive(CD_advection,
                      N=N_slider, 
                      u=u_slider
                     )

display(w)

#CD_advection(5, 2.5)